In [1]:
import io
import re
import json
from glob import glob

https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

http://brandonrose.org/clustering#Latent-Dirichlet-Allocation

https://radimrehurek.com/gensim/tut1.html

Mālikīs:
Unẓr
Madhhab
Rushd
Nawādir
المواز


Shāfīʿī:
روي - and the like


Ḥanbalī:
much more quotes (qāl, ʿan,, qīl)

# LDA

In [1]:
from gensim import corpora
from gensim.models import LdaModel, LdaSeqModel

C:\Users\mayet\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
dict_134 = corpora.Dictionary.load_from_text('gensim_files/dict_134.dict')
dict_135 = corpora.Dictionary.load_from_text('gensim_files/dict_135.dict')
dict_136 = corpora.Dictionary.load_from_text('gensim_files/dict_136.dict')
dict_137 = corpora.Dictionary.load_from_text('gensim_files/dict_137.dict')

In [3]:
corpus_134 = corpora.MmCorpus('gensim_files/corpus_134.mm')
corpus_135 = corpora.MmCorpus('gensim_files/corpus_135.mm')
corpus_136 = corpora.MmCorpus('gensim_files/corpus_136.mm')
corpus_137 = corpora.MmCorpus('gensim_files/corpus_137.mm')

In [ ]:
lda = LdaModel(corpus_134,
               num_topics=10, 
               id2word=dict_134,
               update_every=15,
               chunksize=100,
               passes=10)

In [ ]:
lda.print_topics(num_topics=10, num_words=30)

# LdaSeqModel

In [ ]:
ldaseq = LdaSeqModel(corpus=corpus_134, time_slice=[7, 26, 8, 7], num_topics=3, chunksize=10, passes=10, random_state=3)

C:\Users\mayet\Anaconda3\lib\site-packages\gensim\models\ldaseqmodel.py:290: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)


In [ ]:
ldaseq.print_topics(num_topics=3, num_words=30)

In [ ]:
ldaseq_dict = LdaSeqModel(corpus=corpus_134, id2word=dict_134, time_slice=[7, 26, 8, 7], num_topics=3, chunksize=10, passes=10, random_state=3)

In [ ]:
ldaseq_dict.print_topics(num_topics=3, num_words=30)

In [6]:
def load_book_order(category_id):
    with open('gensim_files/'+str(category_id)+'_book_order.json') as f:
        return json.load(f)
    
print (load_book_order(134))

[[182, '96606'], [189, '14285'], [189, '18264'], [189, '6161'], [189, '6163'], [189, '6164'], [189, '7231'], [370, '16546'], [373, '14335'], [428, '124336'], [428, '17768'], [461, '6178'], [483, '5423'], [540, '7691'], [586, '22362'], [587, '8183'], [593, '11820'], [593, '6170'], [616, '10875'], [666, '6173'], [683, '1066'], [686, '6226'], [710, '14262'], [743, '23023'], [773, '7534'], [786, '9403'], [792, '18259'], [800, '21596'], [803, '11816'], [855, '13784'], [855, '427'], [882, '6179'], [885, '11446'], [956, '6185'], [970, '12227'], [1005, '13620'], [1030, '21639'], [1069, '12117'], [1069, '412'], [1078, '21644'], [1088, '14250'], [1231, '11993'], [1252, '21613'], [1298, '21496'], [1304, '6160'], [1306, '14450'], [1306, '918'], [1340, '8502']]


# TFiDF

In [74]:
import io
from pyarabic import araby
from nltk.corpus import stopwords
from collections import OrderedDict
from stop_words import get_stop_words
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [76]:
sw1 = get_stop_words('arabic') + stopwords.words("arabic")
sw2 = ['ا','أ','إ','ذ','ض','ص','ث','ق','ف','غ','ع','ه','خ','ح','ج','ش','س','ي','ب','ل','ا','ال','ت','ن','م','ك','ئ','ء','ؤ','ر','لا','ى','ة','و','ز','ظ']
sw3 = ["آله", "أبو", "أبي", "أثنى", "أحدهما", "أظهرهما", "أن", "أنه", "أو", "أى", "إلا", "إلخ", "إله", "إلى", "ابن", "ابو","ابي", "الآتي", "الأستاذ", "الأولباب", "الامام", "البصير", "الثانية", "الجلال", "الحمد", "الخ", "الدكتور", "الرحمن", "الرحيم", "الرسول", "السميع","الشارح", "الشيء", "الشيخ", "الصمد", "العبد", "العلامة", "العلي", "الفقير", "القدير", "الكتاب", "الله", "المؤلف", "المؤلفة", "المجلد","المسألة", "المصنف", "النبي", "الي", "انتهى", "انظر", "اهـ", "باب", "بالواو", "بدلالة", "برقم", "بسم", "بعد", "بقيد", "بمثابة", "بن", "به","بها", "بين", "بينهما", "تأمل", "تخريجه", "تعالى", "تعبيره", "ثم", "ثناء", "حتى", "حكاهما", "حمدا", "خصما", "دليلنا", "ذكرنا", "ذكرناه", "ر"]
sw4 = ["فقال","وعن","ربه", "رحمة", "رسول", "رضى", "رضي", "رقم", "رها", "سبحانه", "سنن", "سيأتي", "شرح", "شيخ", "صلى", "طبقات", "عبد", "على", "عليكم", "عليه", "عن","عند", "عنه", "غير", "فأشبه", "فأما", "فإن", "فإنا", "فإنه", "فائدة", "فافهم", "فالأصح", "فالقاضي", "فالوجه", "فان", "فانه", "فجاز", "فدل", "فصل","فكان", "فلأن", "فلأنه", "فلا", "فلما", "فليتأمل", "فمنهم", "فنقول", "فهذا", "فهل", "فوجهان", "فى", "في", "فيه", "فيها", "قال", "قبل", "قدمناه","قلنا", "قول", "قولان", "قوله", "كان", "كتاب", "كلام", "كلامهم", "كما", "كونه", "لأنا", "لأنه", "لأنها", "لان", "لانه", "لخبر", "لذلك", "لرحمة","لقوله", "له", "لهم", "لو", "مادة", "مثال", "مثلا", "محمد", "مع", "معطوف", "مقدمة", "من", "مناهج", "منتهى", "منه", "نسخة", "نسلم", "نصا","نصه", "نقول", "هريرة", "ههنا", "وأصحهما", "وأما", "وأيضا", "وإن", "وإنما", "وإنه", "واحتج", "واعلم", "والتقوى", "والثانى", "والثاني","والسلام", "والصلاة", "وان", "وانظر", "وبالله", "وبه", "وتقدم", "وجزم", "وجل", "وجهان", "وسلم", "وسن", "وشرعا", "وصلى", "وعبارة", "وعلى","وعنه", "وغيره", "وغيرهم", "وفى", "وقال", "وقد", "وقدمه", "وقوله", "وقولي", "وقيل", "وكذلك", "ولأن", "ولأنه", "ولأنها", "ولذا", "ولكنا", "ولنا", "ولو","ومحمد", "وهذا", "وهنا", "وهو", "ويحتمل", "يعني", "يقال", "يقول", "يكون"]
sw = set(sw1+sw2+sw3+sw4)

In [77]:
st = ISRIStemmer()

def not_sw(text):
    return (text not in sw) or st.stem(text) not in sw

def strip_text(text):
    return araby.strip_tatweel(araby.strip_tashkeel(text))

In [ ]:
bow = cv.fit_transform(rs)

sum_words = bow.sum(axis=0)

words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

In [3]:
def dummy_fun(doc):
    return doc

cv = CountVectorizer(
        stop_words=sw)

tfidf = TfidfVectorizer(
        analyzer='word',
        tokenizer=dummy_fun,
        preprocessor=dummy_fun,
        token_pattern=None)

In [4]:
all_tfidfs = []
for category_id in ['134','135','136','137']:
    with open('gensim_files/texts_'+str(category_id)+'.json') as f:
        books = json.load(f)
    
    bow = tfidf.fit_transform(books)
    sum_words = bow.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in tfidf.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    
    all_tfidfs.append(words_freq)

In [7]:
all_tfidfs[1][:50]

[('مالك', 9.485459571233676),
 ('لأن', 8.15825278466134),
 ('القاسم', 7.043596488040608),
 ('الصلاة', 6.830838409892552),
 ('المدونة', 5.691046812722358),
 ('يجوز', 5.365842906800691),
 ('شيء', 4.964737304470881),
 ('فقال', 4.47489802281021),
 ('الإمام', 4.258558002111478),
 ('غيره', 4.2150834569198805),
 ('الأول', 4.155380941231384),
 ('إليه', 3.8097015588337038),
 ('يكن', 3.7484433662034826),
 ('وإلا', 3.7302536387125746),
 ('أيضا', 3.39781398816008),
 ('أهل', 3.3519039813878995),
 ('فهو', 3.3415728900614026),
 ('السلام', 3.2956678294806263),
 ('ذكر', 3.2129995483132774),
 ('المال', 3.1604985600376807),
 ('حكم', 3.0810545514224947),
 ('مثل', 3.078903759808531),
 ('بأن', 2.9231996996246665),
 ('الماء', 2.902725554971079),
 ('بيع', 2.8269349918480953),
 ('صلاة', 2.8237942043520268),
 ('بذلك', 2.804015753234941),
 ('المشهور', 2.7444353663177337),
 ('وليس', 2.74347435951192),
 ('أحد', 2.6977884427437537),
 ('قلت', 2.645999524879351),
 ('خلاف', 2.6429270497166586),
 ('وابن', 2.58919142382

# Get Book Order for DTM

https://stackoverflow.com/questions/43789439/python-json-loads-changes-the-order-of-the-object

In [41]:
from bs4 import BeautifulSoup

In [53]:
page = io.open('../../../../../Desktop/OEBPS/xhtml/P'+str(1)+'.xhtml', 'rb').read()

In [56]:
soup = BeautifulSoup(page, "lxml")

In [57]:
soup.find('div',{'id':'book-container'}).text

'\n\n\nمُقَدّمَة الْمُؤلفبِسم الله الرَّحْمَن الرَّحِيم وَبِه نستعينالْحَمد لله، ونحمده وَنَسْتَعِينُهُ وَنَسْتَغْفِرُهُ، وَنَعُوذُ بِاللَّهِ مِنْ شُرُورِ أَنْفُسِنَا وَمن سيئات أَعمالنَا، من ييهده اللَّهُ فَلَا مُضِلَّ لَهُ، وَمَنْ يُضْلِلْ فَلَا هَادِيَ لَهُ. وَأَشْهَدُ أَنْ لَا إِلَهَ إِلَّا اللَّهُ شَهَادَةً تَكُونُ لِلنَّجَاةِ وَسِيلَةً، وَلِرَفْعِ الدَّرَجَاتِ كَفِيلَةً، وَأَشْهَدُ أَنَّ مُحَمَّدًا عَبْدُهُ وَرَسُولُهُ، الَّذِي بَعثه وطرق الْإِيمَانِ قَدْ عَفَتْ آثَارُهَا، وَخَبَتْ أَنْوَارُهَا، وَوَهَنَتْ أَرْكَانهَا، وَجَهل مَكَانهَا، فشيد صلوَات الله وسلامة عَلَيْهِ من معالمها مَا عَفا، وشفى من الغليل فِي تَأْيِيدِ كَلِمَةِ التَّوْحِيدِ مَنْ كَانَ عَلَى شفى، وأوضح سَبِيل الهادية لِمَنْ أَرَادَ أَنْ يَسْلُكَهَا، وَأَظْهَرَ كُنُوزَ السَّعَادَةِ لِمَنْ قَصَدَ أَنْ يَمْلِكَهَا. أَمَّا بَعْدُ؛ فَإِنَّ التَّمَسُّكَ بِهَدْيهِ لَا يَسْتَتِبُّ إِلَّا بِالِاقْتِفَاءِ لِمَا صَدَرَ مِنْ مِشْكَاتِهِ، وَالِاعْتِصَامُ بِحَبْلِ اللَّهِ لَا يَتِمُّ إِلَّا بِبَيَانِ كَشْفِهِ، وَكَانَ «كِتَابُ ا